In [1]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import plotly.express as px
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import networkx as nx

from statistics import mean
pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: f'%.{2}f' % x)
pd.set_option('display.max_colwidth', None)


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../8Jul2020-14Jul2020.csv')
df['Date'] = pd.to_datetime(df['Start Date'], dayfirst=True)
df.sort_values(by='Date', inplace=True)
df['Date'] = df['Date'].dt.date

cluster = df[['StartStation Name', 'StartStation lat', 'StartStation long']]
cluster.drop_duplicates(keep='first', inplace=True)
cluster.head()

,StartStation Name,StartStation lat,StartStation long
0,"Jubilee Plaza, Canary Wharf",51.50,-0.02
1,"Ford Road, Old Ford",51.53,-0.03
2,"Houndsditch, Aldgate",51.51,-0.08
4,"Ossulston Street, Somers Town",51.53,-0.13
6,"Endsleigh Gardens, Euston",51.53,-0.13


In [3]:
X = cluster[['StartStation lat', 'StartStation long']].values

n_clusters_ = 3
kmeans = KMeans(n_clusters = n_clusters_, init ='k-means++')
kmeans.fit(X) # Compute k-means clustering.
labels = kmeans.fit_predict(X)

# Coordinates of cluster centers.
centers = kmeans.cluster_centers_ 
# print((centers))

# map each station with its region
cluster['clusters'] = labels
cluster.index = cluster['StartStation Name']
cluster_map = cluster['clusters'].to_dict()

df['cluster'] = df['StartStation Name'].map(cluster_map)
df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks,Date,cluster
0,99188805,780,13656,08/07/2020 00:13,476,"Stebondale Street, Cubitt Town",08/07/2020 00:00,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,2020-07-08,2
1,99188806,180,4393,08/07/2020 00:03,531,"Twig Folly Bridge, Mile End",08/07/2020 00:00,517,"Ford Road, Old Ford",51.53,-0.04,6,6,0,15,21,51.53,-0.03,2,2,0,16,18,2020-07-08,2
2,99188809,960,8973,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08,2
3,99188810,960,7141,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08,2
4,99188807,600,15085,08/07/2020 00:11,201,"Dorset Square, Marylebone",08/07/2020 00:01,797,"Ossulston Street, Somers Town",51.52,-0.16,15,15,0,1,16,51.53,-0.13,28,27,1,2,30,2020-07-08,0


In [4]:
sum_duration = {}
mean_duration = {}

for n in range(3,750,3):
    print(n)
    n_clusters_ = n
    kmeans = KMeans(n_clusters = n_clusters_, init ='k-means++')
    kmeans.fit(X) # Compute k-means clustering.
    labels = kmeans.fit_predict(X)

    # Coordinates of cluster centers.
    centers = kmeans.cluster_centers_ 
    # print((centers))

    # map each station with its region
    cluster['clusters'] = labels
    cluster.index = cluster['StartStation Name']
    cluster_map = cluster['clusters'].to_dict()

    df['cluster'] = df['StartStation Name'].map(cluster_map)
    
    mean_cluster = df.groupby(['cluster'])['Duration'].mean().values
    
    flag = sum(mean_cluster)
    sum_duration[n] = flag

    flag = mean(mean_cluster)
    mean_duration[n] = flag    
    
# df.head()

3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
51
54
57
60
63
66
69
72
75
78
81
84
87
90
93
96
99
102
105
108
111
114
117
120
123
126
129
132
135
138
141
144
147
150
153
156
159
162
165
168
171
174
177
180
183
186
189
192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297
300
303
306
309
312
315
318
321
324
327
330
333
336
339
342
345
348
351
354
357
360
363
366
369
372
375
378
381
384
387
390
393
396
399
402
405
408
411
414
417
420
423
426
429
432
435
438
441
444
447
450
453
456
459
462
465
468
471
474
477
480
483
486
489
492
495
498
501
504
507
510
513
516
519
522
525
528
531
534
537
540
543
546
549
552
555
558
561
564
567
570
573
576
579
582
585
588
591
594
597
600
603
606
609
612
615
618
621
624
627
630
633
636
639
642
645
648
651
654
657
660
663
666
669
672
675
678
681
684
687
690
693
696
699
702
705
708
711
714
717
720
723
726
729
732
735
738
741
744
747


In [5]:
mean_cluster_df = pd.DataFrame(list(mean_duration.items()), columns=['cluster', 'mean'])
mean_cluster_df.head()

,cluster,mean
0,3,1613.52
1,6,1623.78
2,9,1607.02
3,12,1618.33
4,15,1616.57


### Bar Chart

In [6]:
fig = px.bar(mean_cluster_df, x='cluster', y='mean')
fig.show()

### Line Chart 

In [7]:
fig = px.line(mean_cluster_df, x='cluster', y='mean')
fig.show()

In [8]:
sum_cluster_df = pd.DataFrame(list(sum_duration.items()), columns=['cluster', 'sum'])
sum_cluster_df.head()

,cluster,sum
0,3,4840.55
1,6,9742.66
2,9,14463.15
3,12,19419.91
4,15,24248.52


### Bar Plot

In [9]:
# fig = px.bar(sum_cluster_df, x='cluster', y='sum')
# fig.show()

### Line Chart

In [10]:
# fig = px.bar(sum_cluster_df, x='cluster', y='sum')
# fig.show()